In [75]:
import requests
import pandas as pd
import polars as pl
import duckdb

from duckdb.typing import VARCHAR
from hashlib import md5
from pathlib import Path

In [76]:
print(
    f"pandas version: {pd.__version__}",
    f"\npolars version: {pl.__version__}",
    f"\nduckdb version: {duckdb.__version__}"
)

pandas version: 2.2.3 
polars version: 1.16.0 
duckdb version: 1.1.3


In [77]:
url_usr = "https://raw.githubusercontent.com/danielmiessler/SecLists/refs/heads/master/Usernames/Names/names.txt"
url_pwd = "https://raw.githubusercontent.com/danielmiessler/SecLists/refs/heads/master/Passwords/500-worst-passwords.txt"

In [78]:
resp_usr = requests.get(url_usr)
resp_pwd = requests.get(url_pwd)

In [79]:
for resp in (resp_usr, resp_pwd):
    assert resp.status_code == 200

In [80]:
path = Path().cwd() / "data.csv"
if not path.exists():

    with open(str(path), mode="w", encoding="utf-8") as f:
        f.write("usr,pwd\n")  # Write the header row
        for usr in filter(None, map(lambda b: b.decode("utf-8").strip(), resp_usr.content.split(b"\n"))):
            for pwd in filter(None, map(lambda b: b.decode("utf-8").strip(), resp_pwd.content.split(b"\n"))):
                f.write(f"{usr},{pwd}\n")


In [81]:
def md5_hash(value: str) -> str:
    return md5(value.encode()).hexdigest()

In [82]:
%%time

pl_dataframe = pl.read_csv(path)

pl_dataframe = pl_dataframe.with_columns(
     (pl.col(["usr", "pwd"]).map_elements(md5_hash, return_dtype=pl.String).name.suffix("_md5"))
)

pl_dataframe.write_csv("eager_polars.csv")  


CPU times: user 6.81 s, sys: 729 ms, total: 7.54 s
Wall time: 6.51 s


In [83]:
%%time

pl_lazyframe = pl.scan_csv(path)

pl_lazyframe = pl_lazyframe.with_columns(
     (pl.col(["usr", "pwd"]).map_elements(md5_hash, return_dtype=pl.String).name.suffix("_md5"))
)

pl_lazyframe.collect().write_csv("lazy_polars.csv")  

CPU times: user 6.85 s, sys: 893 ms, total: 7.74 s
Wall time: 6.55 s


In [84]:
%%time

pd_dataframe = pd.read_csv(path).dropna()

for col in ["usr", "pwd"]:
    pd_dataframe[f"{col}_md5"] = pd_dataframe[col].apply(md5_hash)

pd_dataframe.to_csv("pandas.csv", index=False)

CPU times: user 11.9 s, sys: 775 ms, total: 12.7 s
Wall time: 12.9 s


In [86]:
%%time
# Open DuckDB connection
con = duckdb.connect()

# Register the MD5 function with DuckDB, specifying input and return types
con.create_function("md5_hash", md5_hash, [VARCHAR], VARCHAR)

# Read the CSV file, compute MD5 hashes, and write to a new CSV file
con.execute("""
    COPY (
        SELECT
            usr,
            pwd,
            md5_hash(usr) AS usr_md5,
            md5_hash(pwd) AS pwd_md5
        FROM read_csv_auto('data.csv')
    ) TO 'duckdb.csv' WITH (HEADER, DELIMITER ',');
""")

con.close()


CPU times: user 11.8 s, sys: 780 ms, total: 12.6 s
Wall time: 10.9 s
